In [1]:
dataset_id = "vax_staging"
!bq --location=US mk --dataset {dataset_id}

Dataset 'studied-brand-266702:vax_staging' successfully created.


In [31]:
!bq --location=US load --autodetect --skip_leading_rows=1 --ignore_unknown_values \
--source_format=CSV {dataset_id}.Events 'gs://cs327e_project_data/vaccine_data/2019VAERSDATA.csv'

Waiting on bqjob_r73351cb0724fbcb0_000001702b882b7b_1 ... (6s) Current status: DONE   


In [13]:
!bq --location=US load --autodetect --skip_leading_rows=1 --ignore_unknown_values \
--source_format=CSV {dataset_id}.Symptoms 'gs://cs327e_project_data/vaccine_data/2019VAERSSYMPTOMS.csv'

Waiting on bqjob_r7639dcb5d8025f81_000001702b44f0e2_1 ... (3s) Current status: DONE   


In [14]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.Vaccines 'gs://cs327e_project_data/vaccine_data/2019VAERSVAX.csv'

Waiting on bqjob_r14fe99858ff9d29_000001702b46431c_1 ... (2s) Current status: DONE   


In [32]:
%%bigquery
select * from vax_staging.Events
limit 5

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,794224,2019-01-02,None,51.0,None,None,F,None,Receive second dose of SHINGRIX on 12/28/18. N...,None,...,None,None,None,None,2,2018-12-30,None,None,None,aspirin
1,794226,2019-01-02,None,65.0,None,None,F,None,Received SHINGRIX vaccine (first dose) and the...,None,...,None,High BP; Elevated cholesterol; GERD; No alcoho...,None,None,2,2018-12-29,None,None,None,Statins
2,794359,2019-01-02,OH,18.0,None,None,U,None,"Patient had swelling and redness, but also had...",None,...,None known,Lupus,None,None,2,2019-01-02,None,None,True,Methotrexate
3,794763,2019-01-06,NC,28.0,None,None,F,None,"Fatigue, pain, development of itchy sores on r...",None,...,Productive persistent cough; fatigue,None.,None,None,2,2019-01-06,None,None,None,"Latex, reaction to unknown chemical in lab."
4,795410,2019-01-09,None,NaN,None,None,F,None,Vaccine administered on 5th 24 hours later red...,None,...,None,HTN,None,None,2,2019-01-08,None,None,None,Heparin


In [50]:
%%bigquery
select * from vax_staging.Symptoms where symptom2 is not NULL
limit 5

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,794252,Rash,22.0,Tic,22.0,Urticaria,22.0,None,None,None,None
1,795935,Fatigue,22.0,Tic,22.0,None,NaN,None,None,None,None
2,818880,Throat clearing,22.0,Tic,22.0,None,NaN,None,None,None,None
3,824337,Abnormal behaviour,22.0,Tic,22.0,None,NaN,None,None,None,None
4,825003,Muscle twitching,22.0,Tic,22.0,None,NaN,None,None,None,None


In [17]:
%%bigquery
select * from vax_staging.Vaccines
limit 5

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,801118,HIBV,SANOFI PASTEUR,None,UNK,None,None,HIB (ACTHIB)
1,801239,HIBV,SANOFI PASTEUR,None,UNK,None,None,HIB (ACTHIB)
2,803164,HIBV,SANOFI PASTEUR,UI917AA,UNK,UN,None,HIB (ACTHIB)
3,804902,HIBV,SANOFI PASTEUR,None,UNK,None,None,HIB (ACTHIB)
4,807785,HIBV,SANOFI PASTEUR,None,1,None,None,HIB (ACTHIB)


In [43]:
%%bigquery
select * from vax_staging.Events where died=TRUE and hospital=TRUE and age_yrs is not null order by age_yrs
limit 5
# This query looks at 5 patients that died and also went to the hospital after their adverse event. Youngest 5 shown first

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,800352,2019-02-06,WA,0.17,0.0,0.2,M,None,"Over the next 4 days, showed a lack of smiling...",True,...,None,None,None,None,2,2019-02-06,None,None,None,None
1,847509,2019-11-14,None,0.17,NaN,NaN,M,None,metabolic acidosis; failure to thrive; develop...,True,...,Congenital anomaly; Immunization,None,None,US0095075131911USA003240,2,2019-11-14,None,None,None,None
2,801369,2019-02-13,NC,0.33,0.0,0.3,M,None,Child was in office on February 12th for his f...,True,...,Nasal congestion,None,None,None,2,2019-02-13,None,None,None,NKDA
3,823707,2019-07-16,KY,0.33,0.0,0.3,F,None,Pt's mother states that pt had fever for 24hrs...,True,...,None,None,None,None,2,2019-07-16,None,None,None,None
4,851409,2019-12-06,NJ,0.42,0.0,0.5,M,None,"Dear Officer, My five months old son passed aw...",True,...,1. mild fever - fever about 100 centigrade ( o...,Nothing,None,None,2,2019-12-06,None,None,True,Nothing


In [44]:
%%bigquery
select * from vax_staging.Events where age_yrs<10 and er_visit=TRUE order by sex
limit 5
# This query looks at the children under 10 that visited the ER after their adverse event with females appearing first.

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,796935,2019-01-22,KY,0.50,0.0,0.6,F,2019-01-21,Immunizations on 11/8/18. Mother noticed knot ...,None,...,None,None,None,None,1,None,None,None,None,None
1,805254,2019-03-13,PA,0.08,0.0,0.1,F,2019-03-12,Patient was seen on 3/8/19 at medical practice...,None,...,None,None,None,None,1,None,None,None,None,None
2,803460,2019-02-27,GA,0.17,0.0,0.2,F,2019-02-18,Rash developing 2 days after vaccines. Startin...,None,...,No,None,None,None,1,None,None,None,None,None
3,803247,2019-02-26,LA,0.33,0.0,0.4,F,2019-02-25,"Fussy, restless, no sleep after vaccines. Afte...",None,...,None,None,None,None,1,None,None,None,None,None
4,815644,2019-05-23,TN,5.00,NaN,NaN,F,2019-05-23,"Red, warm to touch, larger area(5in wx2.5in) a...",None,...,None,None,None,None,1,None,None,None,None,None


In [46]:
%%bigquery
select * from vax_staging.Symptoms where symptom1='Nausea' and symptom2='Vomiting'
limit 5
# This query looks at patients that had nausea and vomiting listed as their first two symptoms. Order by not used because nothing to order by
# with just this table alone.

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,803601,Nausea,22.0,Vomiting,22.0,None,None,None,None,None,None
1,804638,Nausea,22.0,Vomiting,22.0,None,None,None,None,None,None
2,805134,Nausea,22.0,Vomiting,22.0,None,None,None,None,None,None
3,807269,Nausea,22.0,Vomiting,22.0,None,None,None,None,None,None
4,811401,Nausea,22.0,Vomiting,22.0,None,None,None,None,None,None


In [49]:
%%bigquery
select * from vax_staging.Vaccines where vax_type='FLUX' and vax_manu!='UNKNOWN MANUFACTURER'
# This query looks at the FLUX flu vaccines that have a listed manufacturer. This is interesting because most of the FLUX vaccines are listed
# without a known manufacturer. Once again order by not used because nothing to order by.

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,842071,FLUX,SANOFI PASTEUR,59TS2,UNK,IM,LA,INFLUENZA (SEASONAL) (NO BRAND NAME)
1,802105,FLUX,SANOFI PASTEUR,UI980AA,5,UN,RA,INFLUENZA (SEASONAL) (NO BRAND NAME)
2,796905,FLUX,SANOFI PASTEUR,None,UNK,UN,UN,INFLUENZA (SEASONAL) (NO BRAND NAME)
